# Compare Documents
# SubQuestionQueryEngine

In [ ]:
!pip install llama-index pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.9/271.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import openai
openai.api_key = 'YOUR_OPENAI_API_KEY'

In [ ]:
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index import set_global_service_context
from llama_index.response.pprint_utils import pprint_response
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

# Load uber and lyft documents

In [ ]:
lyft_docs = SimpleDirectoryReader(input_files=["lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["uber_2021.pdf"]).load_data()

In [ ]:
print(f'Loaded lyft 10-K with {len(lyft_docs)} pages')
print(f'Loaded Uber 10-K with {len(uber_docs)} pages')

Loaded lyft 10-K with 238 pages
Loaded Uber 10-K with 307 pages


# Build indices

In [ ]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)
uber_index = VectorStoreIndex.from_documents(uber_docs)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Basic QA

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)


In [ ]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3)


In [ ]:
response = await lyft_engine.aquery('What is the revenue of Lyft in 2021? Answer in millions with page reference')


In [ ]:
print(response)

The revenue of Lyft in 2021 was $3,208.3 million. (Page reference: 63)


In [ ]:
response = await uber_engine.aquery('What is the revenue of Uber in 2021? Answer in millions, with page reference')


In [ ]:
print(response)

The revenue of Uber in 2021 was $17,455 million. (Page reference: 98)


# For comparing between uber and lyft

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(name='lyft_10k', description='Provides information about Lyft financials for year 2021')
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(name='uber_10k', description='Provides information about Uber financials for year 2021')
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [ ]:
response = await s_engine.aquery('Compare and contrast the customer segments and geographies that grew the fastest')

Generated 4 sub questions.
[lyft_10k] Q: What were the customer segments that grew the fastest for Lyft in 2021?
[uber_10k] Q: What were the customer segments that grew the fastest for Uber in 2021?
[lyft_10k] Q: Which geographies experienced the fastest growth for Lyft in 2021?
[uber_10k] Q: Which geographies experienced the fastest growth for Uber in 2021?
[uber_10k] A: The customer segments that grew the fastest for Uber in 2021 were the Mobility and Delivery segments.
[lyft_10k] A: Lyft's 2021 report does not provide specific information about the geographies that experienced the fastest growth for the company.
[lyft_10k] A: Lyft experienced growth in the number of Active Riders in 2021 compared to the previous year. However, the context does not provide specific information about the customer segments that grew the fastest for Lyft in 2021.
[uber_10k] A: Uber experienced the fastest growth in its Mobility Gross Bookings in five metropolitan areas in 2021. These metropolitan areas 

In [ ]:
print(response)

The customer segments that grew the fastest for Lyft in 2021 are not specified in the given context. However, for Uber, the customer segments that grew the fastest in 2021 were the Mobility and Delivery segments. In terms of geographies, Lyft's 2021 report does not provide specific information about the geographies that experienced the fastest growth. On the other hand, Uber experienced the fastest growth in its Mobility Gross Bookings in five metropolitan areas in 2021, namely Chicago, Miami, New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom.


In [ ]:
response = await s_engine.aquery('Compare revenue growth of Uber and Lyft from 2020 to 2021')

Generated 4 sub questions.
[uber_10k] Q: What was the revenue of Uber in 2020?
[uber_10k] Q: What was the revenue of Uber in 2021?
[lyft_10k] Q: What was the revenue of Lyft in 2020?
[lyft_10k] Q: What was the revenue of Lyft in 2021?
[lyft_10k] A: The revenue of Lyft in 2020 was $2,364,681.
[lyft_10k] A: The revenue of Lyft in 2021 was $3,208,323,000.
[uber_10k] A: The revenue of Uber in 2021 was $17.5 billion.
[uber_10k] A: The revenue of Uber in 2020 was $11,139 million.


In [ ]:
print(response)

The revenue growth of Uber from 2020 to 2021 was significantly higher compared to Lyft.
